In [31]:
import cirq
import numpy as np

def apply_oracle(circuit, qubits, mu, n):
    """Applies the oracle for f(x) = x · mu mod 2"""
    for i in range(n):
        if mu[i] == 1:
            circuit.append(cirq.CNOT(qubits[i], qubits[n])) # CNOT from input i to output qubit if s[i] = 1

def bernstein_vazirani(n=4, mu_input=None):
    """
    Implements the Bernstein-Vazirani algorithm to find the hidden string mu
    n: number of input qubits
    mu_input: hidden string mu (if None, randomly generated)
    """
    if mu_input is None:
        mu_input = np.random.randint(0, 2, n)
    mu = mu_input
    # Create n input qubits and 1 output qubit
    qubits = [cirq.LineQubit(i) for i in range(n + 1)]
    circuit = cirq.Circuit()
    # Step 1: Prepare the output qubit in |-> state (MISSING LINES)
    circuit.append(cirq.X.on(qubits[-1]))
    circuit.append(cirq.H.on(qubits[-1]))
    # Step 2: Apply Hadamard gates to input qubits (MISSING LINE)
    for i in range(n):
        circuit.append(cirq.H.on(qubits[i]))
    # Step 3: Apply the oracle
    apply_oracle(circuit, qubits, mu, n)
    # Step 4: Apply Hadamard gates to input qubits again (MISSING LINE)
    for i in range(n):
        circuit.append(cirq.H.on(qubits[i]))
    # Step 5: Measure the input qubits (MISSING LINE)
    circuit.append(cirq.measure(*qubits[0:-1],key='result'))
    simulator = cirq.Simulator()
    results = simulator.run(circuit)
    measurements = results.measurements['result']
    return circuit, measurements, mu
print("Bernstein-Vazirani Algorithm Homework - Complete the Implementation")
print("==============================================================")
n = 4
mu = [1, 0, 1, 1] # mu = 1011 in binary
circuit, measurements, mu = bernstein_vazirani(n, mu)
print(f"Circuit:\n{circuit}")
print(f"Measurement result: {measurements}")
print(f"Actual mu: {mu}")

Bernstein-Vazirani Algorithm Homework - Complete the Implementation
Circuit:
0: ───H───────@───H───────────M('result')───
              │               │
1: ───H───H───┼───────────────M─────────────
              │               │
2: ───H───────┼───@───H───────M─────────────
              │   │           │
3: ───H───────┼───┼───@───H───M─────────────
              │   │   │
4: ───X───H───X───X───X─────────────────────
Measurement result: [[1 0 1 1]]
Actual mu: [1, 0, 1, 1]


In [76]:
# %%
import cirq
import numpy as np

# %%
def apply_oracle(circuit, input_qubits, output_qubits, latent_a, n):
    """Applies the oracle f(x) = f(x ⊕ s) without using a class"""
    # For simplicity, implement f(x) as x ⊕ s for x < 2^(n-1)
    # and map the other half accordingly
    for i in range(n):
        circuit.append(cirq.CNOT(input_qubits[i], output_qubits[i]))
        
    for i in range(n):
        if latent_a[i] == 1:
            circuit.append(cirq.X(output_qubits[i]))
            
# %%
def simons_algorithm(n=3, latent_ao=None):
    """
    Implements Simon's algorithm to find the hidden period s
    n: number of qubits
    latent_a: hidden period (if None, randomly generated)
    """
    # Step 1: Set up the latent_a if not provided
    if latent_ao is None:
        latent_ao = np.random.randint(0, 2, n)
    latent_a = latent_ao # Avoid shadowing issues
    # Step 2: Create quantum circuit
    input_qubits = [cirq.LineQubit(i) for i in range(n)]
    output_qubits = [cirq.LineQubit(i) for i in range(n, 2*n)]
    circuit = cirq.Circuit()
    # Step 3: Apply Hadamard gates to input qubits
    for i in range(n):
        circuit.append(cirq.H.on(input_qubits[i]))
    # Step 4: Apply the oracle
    apply_oracle(circuit, input_qubits, output_qubits, latent_ao, n)
    # Step 5: Apply Hadamard gates again
    for i in range(n):
        circuit.append(cirq.H.on(input_qubits[i]))
    # Step 6: Measure
    circuit.append(cirq.measure(*input_qubits, key='result'))
    # Step 7: Simulate the circuit
    simulator = cirq.Simulator()
    results = simulator.run(circuit, repetitions=n) # Run n times
    # Step 8: Collect measurement results
    measurements = results.measurements['result']
    print(f"Circuit:\n{circuit}")
    print(f"Measurement results:\n{measurements}")
    print(f"Actual latent_a: {latent_a}")
    return circuit, measurements, latent_a
    
# %%
print("Simon's Algorithm - Complete Implementation")
print("==========================================")

# Run the algorithm with 3 qubits and a specific latent_a
n = 3
latent_a = [1, 0, 1] # s = 101 in binary
circuit, measurements, latent_a = simons_algorithm(n, latent_a)

print("\nVerification:")
print("Check that each measurement y satisfies y · a = 0 (mod 2)")
print("For a = [1,0,1]:")
for i, y in enumerate(measurements):
    dot_product = sum(y[j] * latent_a[j] for j in range(n)) % 2
    print(f"y[{i}] = {y}, y · a = {dot_product} (mod 2)")
    
# %%

Simon's Algorithm - Complete Implementation
Circuit:
          ┌───┐
0: ───H────@──────H───M('result')───
           │          │
1: ───H────┼@─────H───M─────────────
           ││         │
2: ───H────┼┼@────H───M─────────────
           │││
3: ────────X┼┼────X─────────────────
            ││
4: ─────────X┼──────────────────────
             │
5: ──────────X────X─────────────────
          └───┘
Measurement results:
[[0 1 1]
 [1 1 0]
 [1 1 1]]
Actual latent_a: [1, 0, 1]

Verification:
Check that each measurement y satisfies y · a = 0 (mod 2)
For a = [1,0,1]:
y[0] = [0 1 1], y · a = 1 (mod 2)
y[1] = [1 1 0], y · a = 1 (mod 2)
y[2] = [1 1 1], y · a = 0 (mod 2)
